# Fermionic Dimer

In [ ]:
using KadanoffBaym
using LinearAlgebra
using LsqFit
using JLD

using PyPlot
using PyCall
qt = pyimport("qutip")
np = pyimport("numpy")

font_size = 16
# font_size = 20

PyPlot.matplotlib.rc("text", usetex=true)
PyPlot.matplotlib.rc("font", family="serif", size=font_size)
PyPlot.matplotlib.rc("axes", labelsize=font_size)
PyPlot.matplotlib.rc("xtick.major", size=8)
PyPlot.matplotlib.rc("ytick.major", size=8)
PyPlot.matplotlib.rc("xtick.minor", visible=true, size=4)
PyPlot.matplotlib.rc("ytick.minor", visible=true, size=4)
PyPlot.matplotlib.rc("xtick", top=true, direction="inout")
PyPlot.matplotlib.rc("ytick", right=true, direction="inout")

## Model

### Non-Hermitian Hamiltonian:

$$
\begin{align}\begin{split}
    \hat{H} &= \varepsilon_1 c^{\dagger}_1 c^\phantom{\dagger}_1  + \varepsilon_2 c^{\dagger}_2 c^\phantom{\dagger}_2 + J \left(c^{\dagger}_1 c^\phantom{\dagger}_2 + c^{\dagger}_2 c^\phantom{\dagger}_1\right)
\end{split}\end{align}
$$


### Equations of motion

#### Vertical Time.

\begin{align}\begin{split}
    0 &= \begin{pmatrix}
    i \partial_t - \varepsilon_1  & -J \\
    -J & i \partial_t - \varepsilon_2
    \end{pmatrix} 
    \begin{pmatrix}
    G^<_{11} & G^<_{12} \\
    G^<_{21} & G^<_{22}
    \end{pmatrix}(t, t')  \\
    0 &= \begin{pmatrix}
    i \partial_t - \varepsilon_1  & -J \\
    -J & i \partial_t - \varepsilon_2
    \end{pmatrix} 
    \begin{pmatrix}
    G^>_{11} & G^>_{12} \\
    G^>_{21} & G^>_{22}
    \end{pmatrix}(t, t')   
\end{split}\end{align}

#### Horizontal Time.

\begin{align}\begin{split}
    0 &= \begin{pmatrix}
    G^<_{11} & G^<_{12} \\
    G^<_{21} & G^<_{22}
    \end{pmatrix}(t, t') 
    \begin{pmatrix}
    i \partial_{t'} + \varepsilon_1  & J \\
    J & i \partial_{t'} + \varepsilon_2
    \end{pmatrix}  \\
    0 &= \begin{pmatrix}
    G^>_{11} & G^>_{12} \\
    G^>_{21} & G^>_{22}
    \end{pmatrix}(t, t') 
    \begin{pmatrix}
    i \partial_{t'} + \varepsilon_1  & J \\
    J & i \partial_{t'} + \varepsilon_2
    \end{pmatrix}  
\end{split}\end{align}

#### Equal-Time.

\begin{align}\begin{split}
    0 &= i \partial_T
    \begin{pmatrix}
    G^<_{11} & G^<_{12} \\
    G^<_{21} & G^<_{22}
    \end{pmatrix}(T, 0)
    - \left[\begin{pmatrix}
    \varepsilon_1 & J \\
    J & \varepsilon_2
    \end{pmatrix}, 
    \begin{pmatrix}
    G^<_{11} & G^<_{12} \\
    G^<_{21} & G^<_{22}
    \end{pmatrix}(T, 0)\right]    \\
    0 &= i \partial_T
    \begin{pmatrix}
    G^>_{11} & G^>_{12} \\
    G^>_{21} & G^>_{22}
    \end{pmatrix}(T, 0)  
    - \left[\begin{pmatrix}
    \varepsilon_1 & J \\
    J & \varepsilon_2
    \end{pmatrix}, 
    \begin{pmatrix}
    G^>_{11} & G^>_{12} \\
    G^>_{21} & G^>_{22}
    \end{pmatrix}(T, 0)\right]
\end{split}\end{align}

## Solving

In [ ]:
# final time
T = 5.0

# Hamiltonian
ε₁ = 1.0
ε₂ = -1.0
J = 20.0
H = ComplexF64[ε₁ J; J ε₂];

In [ ]:
# quantum numbers
dim = 2

# Define your Green functions at (t0, t0), time-arguments at the end!
GL = GreenFunction(zeros(ComplexF64, dim, dim, 1, 1), Lesser)
GG = GreenFunction(zeros(ComplexF64, dim, dim, 1, 1), Greater)

# initial condition
N_0 = 1.0

GL[1,1,1,1] = 1.0im * N_0
# gfL[1,2,1,1] = 1.0im * N_0
# gfL[2,1,1,1] = 1.0im * N_0
GG[:, :, 1, 1] = -1.0im .* diagm([1.0, 1.0]) .+ GL[:, :, 1, 1]

u0 = [GL, GG]

function f(u, times, t, t′)
    GL, GG = u[1], u[2]
    f_GL = -1.0im * H * GL[t, t′]
    f_GG = -1.0im * H * GG[t, t′]
    return [f_GL, f_GG]
end

f_diag = (u, times, t) -> f(u, times, t, t) - adjoint.(f(u, times, t, t))

# function f_diag(u, times, t)
#     GL, GG = u[1], u[2]
#     f_GL = -1.0im * (H * GL[t, t] - GL[t, t] * adjoint(H))
#     f_GG = -1.0im * (adjoint(H) * GG[t, t] - GG[t, t] * H)
#     return [f_GL, f_GG]
# end

In [ ]:
# Basically this will mutate Lesser and Greater in place
sol = kbsolve(f, f_diag, u0, (0.0, T); atol=1e-8, rtol=1e-6);

In [ ]:
# stop = Int(n/2) + 1 

times = sol.t #range(0, length=n + 1, stop=T) |> collect;

n = length(sol.t) - 1

horizontal_times = k -> vcat(times[k:end] .- times[k], times[end] .+ (1:(k - 1) |> collect) .* (times[end] - times[end-1]));

In [ ]:
n

## Example plots

In [ ]:
idx1 = 1
idx2 = 1;

In [ ]:
# Analytic result
f_2(T, tp) = exp(-1.0im * H * horizontal_times(T)[tp]) * exp(-1.0im * H * sol.t[T]) * GL[:, :, 1, 1] * exp(1.0im * H * sol.t[T])
ana(T, tp) = exp(-1.0im * H * sol.t[T]) * GL[:, :, 1, 1] * exp(1.0im * H * horizontal_times(T)[tp]) * exp(1.0im * H * sol.t[T])
ana_diag(T) = exp(-1.0im * H * sol.t[T]) * GL[:, :, 1, 1] * exp(1.0im * H * sol.t[T])

In [ ]:
xpad = 8
ypad = 5

figure(figsize=(7, 3))

ax = subplot(121)
plot(J .* sol.t, [imag(ana_diag(k)[idx1, idx2]) for k in 1:n+1], c="C0", ls="--", lw=3.0, alpha=0.25)
plot(J .* sol.t, [imag(GL.data[idx1, idx2, k, k]) for k in 1:length(sol.t)], marker="", ms=3.0, ls="-", c="C0")
# plot(sol.t, [imag(gfL.data[2, 2, k, k]) for k in 1:length(sol.t)], marker="", ms=3.0, ls="-", c="r")
xlim(0, J * 1)
ylim(0, N_0)
xticks(J .* [0, 0.5, 1])
yticks([0, 0.5, 1])
xlabel("\$J t\$")
ylabel("\$\\mathrm{Im}\\; G^<_{11}(t, t)\$")
ax.xaxis.set_tick_params(pad=xpad)
ax.yaxis.set_tick_params(pad=ypad)
ax.set_axisbelow(false)
ticklabel_format(axis="y", style="sci", scilimits=(-0, 0))

ax = subplot(122)
plot(J .* sol.t, [imag.(GL.data[idx1, idx2, k, k]) for k in 1:length(sol.t)] 
    .- [imag.(ana_diag(k)[idx1, idx2]) for k in 1:n+1], marker="", ms=3.0, ls="-", c="r")
xlim(0, J * 5)
# ylim(-0.1, 0.1)
xticks(J .* [0, 1, 2, 3, 4, 5])
xlabel("\$J t\$")
ylabel("\$\\mathrm{Im}\\left[ G^<_{11}(t, t) - \\mathcal{G}^<_{11}(t, t) \\right]\$", labelpad=16)
ax.xaxis.set_tick_params(pad=xpad)
ax.yaxis.set_tick_params(pad=ypad)
ax.yaxis.set_label_position("right")
ax.set_axisbelow(false)
# ax.yaxis.tick_right()
ticklabel_format(axis="y", style="sci", scilimits=(-0, 0))

tight_layout(pad=0.2, w_pad=0, h_pad=0)
savefig("fermion_example_T.pdf")

In [ ]:
xpad = 8
ypad = 5

figure(figsize=(7, 3))

ax = subplot(121)
idx = 1
plot(sol.t[idx:end - 1], [imag(GL.data[idx1, idx2, idx, k + idx - 1]) for k in 1:length(sol.t) - idx], "o", ms=2, c="C0")
plot(sol.t[idx:end - 1], [imag(ana(idx, k))[idx1, idx2] for k in 1:length(sol.t) - idx], "x", c="C0", lw=5.0, alpha=0.5, ms=5)
xlim(0, 1)
ylim(0, N_0)
xticks([0, 0.5, 1])
yticks([-1, -0.5, 0, 0.5, 1])
xlabel("\$t'\$")
ylabel("\$\\mathrm{Im}\\; G^<_{11}(t, t)\$")
ax.xaxis.set_tick_params(pad=xpad)
ax.yaxis.set_tick_params(pad=ypad)
ticklabel_format(axis="y", style="sci", scilimits=(-0, 0))

ax = subplot(122)
idx = 1
plot(sol.t[idx:end - 1], [abs.(GL.data[idx1, idx2, idx, k + idx - 1]) for k in 1:length(sol.t) - idx]
    .- [abs.(ana(idx, k))[idx1, idx2] for k in 1:length(sol.t) - idx], "-", ms=1, c="r")
# xlim(0, 1)
# ylim(-0.1, 0.1)
# xticks([0, 0.5, 1])
# yticks([-0.1, -0.05, 0, 0.05, 0.1])
xlabel("\$t'\$")
ax.xaxis.set_tick_params(pad=xpad)
ax.yaxis.set_tick_params(pad=ypad)
ticklabel_format(axis="y", style="sci", scilimits=(-0, 0))

tight_layout(pad=0.25, w_pad=0, h_pad=0)
# savefig("fermion_example_tp.pdf")

## Error scaling

In [ ]:
epsilons = [2.0^(-k) for k in 10:45]
init_dt = 1e-16; # epsilons[end]
err_data = [(0.0, 0.0, 0.0) for _ in 1:length(epsilons)]
p_norm = 1
for (k, eps) in enumerate(epsilons)
    print(k, ", ")
    
    sol = kbsolve(f, f_diag, u0, (0.0, T); dtini=init_dt, atol=eps*2.0^(-4), rtol=eps);
    n = length(sol.t) - 1;
    
    # Analytic result
    horizontal_times = x -> vcat(sol.t[x:end] .- sol.t[x], sol.t[end] .+ (1:(x - 1) |> collect) .* (sol.t[end] - sol.t[end-1]));
    ana(T, tp) = exp(-1.0im * H * sol.t[T]) * GL[:, :, 1, 1] * exp(1.0im * H * horizontal_times(T)[tp]) * exp(1.0im * H * sol.t[T])
    ana_diag(T) = exp(-1.0im * H * sol.t[T]) * GL[:, :, 1, 1] * exp(1.0im * H * sol.t[T])
    
    horizontal_err = 0.0
    for idx in 1:n
        horizontal_err += norm([(GL.data[1, 1, idx, k + idx]) for k in 1:length(sol.t) - idx] 
            .- [(ana(idx, k))[1, 1] for k in 1:length(sol.t) - idx + 1][2:end], p_norm) # not counting the diagonal
    end    

    err = norm([(GL.data[1, 1, k, k]) for k in 1:length(sol.t)] .- [(ana_diag(k)[1, 1]) for k in 1:n+1], p_norm)
    err += 2 * horizontal_err
    err_data[k] = (n, (1.0/n)^2 * err, eps)
#     err_data[k] = (n, (1.0/n) * err, eps)
end

save("error_data.jld", "params", [T, H, N_0], "err_data", err_data)

In [ ]:
# h5open("error_data.h5", "w") do file
#     params = create_group(file, "params") # create a group
#     params["T"] = T 
#     params["H"] = H
#     params["N_0"] = N_0
#     write.(file, "err_data", err_data)
# end;

# params = h5open("error_data.h5", "r") do file
#     read(file, "params")
# end

In [ ]:
err_data = load("error_data.jld")["err_data"]
err_data = err_data[3:end - 2];

In [ ]:
xdata = log10.([x[1] for x in err_data])
ydata = log10.([x[2] for x in err_data]);

In [ ]:
func_power = (n, p) -> 10^(p[2]) * n^(-p[1]);
fit_func = (n, p) -> -p[1] .* n .+ p[2];
fit_result = curve_fit(fit_func, xdata, ydata, [2.0, 1]);
coef(fit_result)

In [ ]:
figure(figsize=(7, 3))

ax = subplot(121)
plot(xdata, map(x -> fit_func(x, coef(fit_result)), xdata), "--k", lw=2,
     label="\$\\mathcal{O}(h^{"*string(coef(fit_result)[1] |> x -> floor(x, sigdigits=4))*"})\$")
plot(xdata, ydata, "o", lw=0, ms=6,
     markerfacecolor="C0", markeredgewidth=0.25, markeredgecolor="#2D5FAA")
xlim(2.5, 3.25)
ylim(-10, -0)
# ax.set_xticks([2.5, 2.75, 3.0, 3.25])
# ax.set_yticks([0, -5, -10, -15])
xlabel("\$\\log(n)\$")
ylabel("\$\\log(\\epsilon_{\\mathrm{abs}})\$")
legend(loc="best", handlelength=1.8, frameon=false, borderpad=0, labelspacing=0)

ax = subplot(122)
# plot(log10.([x[3] for x in err_data]), log10.([x[1] for x in err_data]), "s")
plot(log10.([x[3] for x in err_data]), log10.([x[1] for x in err_data]), "o", ms=5,
     markerfacecolor="C0", markeredgewidth=0.25, markeredgecolor="#2D5FAA")
xlim(-2, -14)
# ylim(-13, -3)
ax.set_xticks([-2, -6, -10, -14])
# ax.set_yticklabels([])
ax.yaxis.set_label_position("right")
xlabel("\$\\log(\\texttt{rtol})\$")
ylabel("\$\\log(n)\$", labelpad=16)

tight_layout(pad=0.1, w_pad=0.75, h_pad=0)
savefig("error_scaling_fermions.pdf")

## Testing